In [4]:
import numpy as np
import pandas as pd
import seaborn as sn
import re
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from wordcloud import WordCloud



In [5]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
LABR = pd.read_csv(r'/content/gdrive/MyDrive/Final_Dataset.tsv',delimiter='\t')

In [7]:
data = LABR

"""**printing the first 3 rows of the data**"""

data.head(3)

,Label,Text
0,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك تد...
1,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
2,1,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...


In [8]:
print("Data contient {} lignes et {} colonnes.".format(data.shape[0], data.shape[1]))

Data contient 63256 lignes et 2 colonnes.


In [9]:
data.isnull().sum()

Label    0
Text     0
dtype: int64

In [10]:
print("On a  {} doublons dans Data.".format(data.duplicated().sum()))

data.drop_duplicates(inplace = True)

print("On a  {} doublons dans Data.".format(data.duplicated().sum()))

On a  3030 doublons dans Data.
On a  0 doublons dans Data.


In [11]:
positive_reviews = data[data["Label"] > 3]
positive_reviews["sentiment"] = 1

negative_reviews = data[data["Label"] < 3]
negative_reviews["sentiment"] = 0

data = pd.concat([positive_reviews, negative_reviews], ignore_index = True)


<ipython-input-11-7a9fa9191a23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_reviews["sentiment"] = 1
<ipython-input-11-7a9fa9191a23>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews["sentiment"] = 0


In [12]:
data.head(3)
df=data

In [13]:
def preprocessing(x):
    x = re.sub('@[^\s]+', ' ', x)
    x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',x)

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030""]+", flags=re.UNICODE)
    emoji_pattern.sub(r'', x)

    ar_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ#'''
    en_punctuations = string.punctuation
    punctuations = ar_punctuations + en_punctuations
    x = x.translate(str.maketrans('', '', punctuations))

    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    x = re.sub(arabic_diacritics, '', str(x)) 

#     x = re.sub("[إأآا]", "ا", x)
#     x = re.sub("ى", "ي", x)
#     x = re.sub("ة", "ه", x)
#     x = re.sub("گ", "ك", x)
#     x = re.sub(r'(.)\1+', r'\1', x)

    return x


In [14]:
data["Clean_reviews"] = data.Text.apply(lambda x: preprocessing(str(x)))

In [15]:
print('- Avant le prétraitement \n\n',data["Text"][7])
print("\n-----------------------------------------------\n")
print('- Après le prétraitement \n\n',data["Clean_reviews"][7])

- Avant le prétraitement 

 إثر دعوة كريمة من أحد الأصدقاء، قرأتُ رواية "عزازيل" الصادرة عن دار الشروق سنة 2008، وهي من تأليف الدكتور "يوسف زيدان" الخبير في المخطوطات والفلسفة، والفائزة بالجائزة العالمية للرواية العربية كأفضل رواية عربية لعام 2009م. لكني بعد قرائتها أصبحتُ أخاف منها. ليس لأنها تناولت الخلافات اللاهوتية المسيحية القديمة حول طبيعة المسيح ووضع السيدة العذراء، والاضطهاد الذي قام به المسيحيون ضد الوثنيين المصريين في الفترات التي أضحت فيها المسيحية ديانة الأغلبية المصرية، لكن لتوغلها الدقيق في وصف لحظات ضعف هيبا، وتجرده من ثوب رهبانيته! دار حوار حول ذلك مع صديقي، فتبانيت آراؤنا. هو يرى "هيبا" منافقًا، وأنا أراه تجسيدًا لحقيقتنا المتناقضة التي نخجل من الحديث عنها. قال صاحبي: كان عليه أن يختار بين الرهبنة أو التلذذ بأوكتافيا وبعدها مرتا، فقلتُ: ومن منا لا يرتدي ثوب الرهبنة على قلب ذئب. أتحدث هنا عن الطبيعة البشرية سريعة التقلب والتغير من حال لحال، و (من كان منكم بلا خطئية فليرمِ هيبا بحجر)! على كل حال أوقفتني كثيرًا جودة لغتها، وحسن اختيار "زيدان" لألفاظها، وتطرقه لأوصاف لم أس

In [16]:
X = data.Clean_reviews
y = data.sentiment

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state = 42)

"""**printing the number of the train set and the test set**"""

print('Train set', X_train.shape)
print('Test set', X_test.shape)


Train set (38815,)
Test set (9704,)


In [18]:
featurizer = CountVectorizer(decode_error='ignore')
Xtrain = featurizer.fit_transform(X_train)
Xtest = featurizer.transform(X_test)

In [19]:
model = MultinomialNB()
model.fit(Xtrain, y_train)
print("train acc:", model.score(Xtrain, y_train))
print("test acc:", model.score(Xtest, y_test))

train acc: 0.9325518485121731
test acc: 0.8684047815333883
